In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

from keras import backend as K 

import matplotlib.pyplot as plt

K.clear_session()

In [ ]:
from keras import backend as K
K.clear_session()

### Setting Up GPU as a training device

In [ ]:
physical_devices = tf.config.list_physical_devices()

print(physical_devices)

In [ ]:
tf.config.set_visible_devices(physical_devices[1], 'GPU')

### Loading Data

In [ ]:
N_TRAIN_EXAMPLES = 20000
N_TEST_EXAMPLES = 5000

CLASSES=10

In [ ]:
from sklearn.model_selection import train_test_split

(X_train, Y_train), (X_test, Y_test) = keras.datasets.cifar10.load_data()

# Concatenate train and test images
X = np.concatenate((X_train, X_test))
y = np.concatenate((Y_train, Y_test))

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=N_TRAIN_EXAMPLES, test_size=N_TEST_EXAMPLES, random_state=1)

# Getting dummy variables
y_train_fixed = np.zeros((y_train.shape[0], 10))
y_test_fixed = np.zeros((y_test.shape[0], 10))

i = 0
for [val] in y_train:
  y_train_fixed[i][val] = 1
  i += 1
  
i = 0
for [val] in y_test:
  y_test_fixed[i][val] = 1
  i += 1

### Setting up the model

AlexNet implementation in keras:

```py
  keras.Input(shape=(227, 227, 3)),
  layers.Conv2D(96, kernel_size=11, strides=4, padding='same', activation="relu"),
  layers.MaxPooling2D(pool_size=3, strides=2),
  layers.Conv2D(256, kernel_size=5, padding='same', activation="relu"),
  layers.MaxPooling2D(pool_size=3, strides=2),
  layers.Conv2D(384, kernel_size=3, padding='same', activation="relu"),
  layers.Conv2D(384, kernel_size=3, padding='same', activation="relu"),
  layers.Conv2D(256, kernel_size=3, padding='same', activation="relu"),
  layers.MaxPooling2D(pool_size=3, strides=2),
  layers.Dropout(0.5),
  layers.Dense(4096, activation="relu"),
  layers.Dropout(0.5),
  layers.Dense(4096, activation="relu"),
  layers.Dense(CLASSES, activation="softmax")
```

In [ ]:
model = keras.Sequential(
  [
    keras.Input(shape=(32, 32, 3)),
    layers.Conv2D(48, kernel_size=4, strides=2, padding='same', activation="relu"),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Conv2D(128, kernel_size=5, padding='same', activation="relu"),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Conv2D(192, kernel_size=3, padding='same', activation="relu"),
    layers.Conv2D(192, kernel_size=3, padding='same', activation="relu"),
    layers.Conv2D(128, kernel_size=3, padding='same', activation="relu"),
    layers.MaxPooling2D(pool_size=3, strides=2),
    layers.Dropout(0.5),
    layers.Dense(2048, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(2048, activation="relu"),
    layers.Dense(CLASSES, activation="softmax"),
    layers.Flatten()
  ]
)

model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

### Training model

In [ ]:
%%time
batch_size = 256 # Training data will be split in chunks of this size that will be passed through the network at the same time
epochs = 80 # During one epoch the model 'sees' all of the training data, with this parameter we determine how many times the model will be allowed to see them
validation_split = 0.2 # Determines what percentage of training data will be used to validate the model after each epoch
history = model.fit(X_train, y_train_fixed, batch_size=batch_size, epochs=epochs, validation_split=validation_split)

In [ ]:
# Plotting the loss curve
plt.figure(figsize=[6, 4])
plt.plot(history.history['loss'], 'black', linewidth=2.0)
plt.plot(history.history['val_loss'], 'blue', linewidth=2.0)
plt.legend(['Training Loss', 'Validation Loss'], fontsize=12)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Loss', fontsize=10)
plt.title('Loss Curves', fontsize=12)

In [ ]:
# Plotting the accuracy curve
plt.figure(figsize=[6, 4])
plt.plot(history.history['accuracy'], 'black', linewidth=2.0)
plt.plot(history.history['val_accuracy'], 'blue', linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=12)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Accuracy', fontsize=10)
plt.title('Accuracy Curves', fontsize=12)

### Saving model to file

In [ ]:
model.save("models/cifar10_test_model.keras")